## Initializing Pynq

In [ ]:
# importing libraries

# basic libraries
import numpy as np
import scipy
from scipy.signal import spectrogram
import wave
import matplotlib
import matplotlib.pyplot as plt

# custom functions
import common_pynq as com

# threading
from threading import Timer
from time import sleep, perf_counter

# for profiling
import cProfile
import pstats
import io
import os

In [ ]:
# checking audio

from pynq import Overlay
overlay = Overlay('design_1_wrapper.bit')
overlay.audio = overlay.audio_codec_ctrl_0
dir(overlay.audio)

In [ ]:
# configuring audio codec
# note: can be stuck, consider restarting if not finished in 3 mins
# dont run other lines until this is done

overlay.audio.configure()

In [ ]:
# selecting input and checking sample rate
# note: codec is fixed at 48kHz

print("selecting input")
overlay.audio.select_microphone()
sample_rate = overlay.audio.sample_rate
print("sample rate: ")
print(sample_rate)

In [ ]:
# testing microphone input

print("make some noise!")
overlay.audio.bypass(seconds=5)

## Testing NN

In [ ]:
# checking nn with array of zeros

# matching nn input shape
data = np.zeros([1,64])

from axi_stream_driver import NeuralNetworkOverlay
nn = NeuralNetworkOverlay('design_1_wrapper.bit', np.shape(data), np.shape(data))
predictions, latency, throughput = nn.predict(data, profile=True)
errors = np.mean(np.square(data - predictions))
print(errors)

In [ ]:
# testing recording format

print("recording")
overlay.audio.record(7)

# downsampling from 48kHz stereo to 16kHz mono
# refer to https://github.com/Xilinx/PYNQ/blob/master/pynq/lib/_pynq/_audio/audio_adau1761.cpp

data = overlay.audio.buffer[::6]
data[data > 2**23] = data[data > 2**23] - 2**24
data = data/2**24
    
print(data[0:10])
print(np.shape(data))

In [ ]:
# testing live inference without threading (initalizing and updating)

def init_rec():
    
    # downsampling from 48kHz stereo to 16kHz mono
    # refer to https://github.com/Xilinx/PYNQ/blob/master/pynq/lib/_pynq/_audio/audio_adau1761.cpp
    overlay.audio.record(102912/16000)
    data = overlay.audio.buffer[::6]
    data[data > 2**23] = data[data > 2**23] - 2**24
    return data/2**24

def update_rec():

    # downsampling from 48kHz stereo to 16kHz mono
    # refer to https://github.com/Xilinx/PYNQ/blob/master/pynq/lib/_pynq/_audio/audio_adau1761.cpp
    overlay.audio.record(3584/16000)
    data = overlay.audio.buffer[::6]
    data[data > 2**23] = data[data > 2**23] - 2**24
    return data/2**24

def prep(data):
    
    return com.data_to_vector_array(data=data,
                                    n_mels=128,
                                    frames=5,
                                    n_fft=1024,
                                    hop_length=512,
                                    downsample=True,
                                    input_dim=64)

def pred(data):
    
    predictions = np.zeros(np.shape(data))
    for i in range(len(data)):
         predictions[i, :] = nn.predict(data[i, :])
    return predictions

def score(data, predictions):
    
    errors = np.mean(np.square(data - predictions), axis=1)
    y_pred = np.mean(errors)
    if y_pred < 446.041404:
        print("normal,    anomaly score: %f" %y_pred)
    else:
        print("anomalous, anomaly score: %f" %y_pred)

def init():
    temp = init_rec()
    data = prep(temp)
    predictions = pred(data)
    score(data=data, predictions=predictions)
    
def update():
    temp = update_rec()
    data = prep(temp)
    predictions = pred(data)
    score(data=data, predictions=predictions)

with cProfile.Profile() as pr1:
    init()  
with cProfile.Profile() as pr2:
    update()
    
if os.path.exists('nn_audio_init.prof'):
    print('please remove profile file before rerunning')
else:
    pr1.dump_stats('nn_audio_init.prof')
    pr2.dump_stats('nn_audio_update.prof')
    
# to check performance using snakeviz
# export out nn_audio_init.prof and nn_audio_update.prof
# then pip install snakeviz and snakeviz FILENAME.prof

## Threading

In [ ]:
# trying threading on pynq
# initializing timer object from threading

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.profile = kwargs.get('rt_profile', False)
        if self.profile:
            self.times = [perf_counter()]
            del self.kwargs['rt_profile']
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            if self.profile:
                self.times.append(perf_counter())
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [ ]:
# now lets try just playing the audio back to see if it works

def playback(i):
    """
    playback function
    
        catures i frame(s) and plays it back immediately
    """
    overlay.audio.bypass(seconds=i/48000)
    
# create timer object that playback audio every 0.0000625s (16000 Hz)
rt0 = RepeatedTimer(0.0000625, playback, 1)

try:
    sleep(5)
finally:
    rt0.stop()

In [ ]:
# now lets just try plotting the audio buffer in realtime (WIP)

def capture(buffer, i):
    '''
    capture function 
    
        captures 1 frame and store in rolling buffer every 0.0000625s (16000 Hz)
    
    args:
    buffer = rolling buffer containing last 1024 frames
     i = numbers of frames
    '''
    overlay.audio.record(i/48000)
    buffer[:-i] = buffer[i:]
    for j in range(i):
        if overlay.audio.buffer[-j] > 2**23:
            buffer[-j] = (float) (overlay.audio.buffer[-j] - 2**24) / 2**24
        else:
            buffer[-j] = (float) overlay.audio.buffer[-j] / 2**24
        
            
def plot_it(buffer):
    '''
    plots_it function
        
        plots buffer every 1 second
    '''
    data = buffer
    spectrogram = com.log_mel_spectrogram(data=data,
                                          audio_sample_rate=16000,
                                          log_offset=0.0,
                                          hop_length=512,
                                          fft_length=1024,
                                          n_mels=128)
    

#create and fill buffer
buffer = np.zeros(10000)
capture(buffer, 10000)

# create timer object that calls on capture every 0.1s (16000 Hz)
rt0 = RepeatedTimer(0.0000625, capture, buffer, 1)
rt1 = RepeatedTimer(1, plot_it, buffer)

try:
    sleep(5)
finally:
    rt0.stop()
    rt1.stop()
    plt.show()

In [ ]:
# now lets try getting a live anormaly score!

# initalizing threading functions and objects

def prep(data):
    
        return com.log_mel_spectrogram(data=data,
                                       audio_sample_rate=16000,
                                       log_offset=0.0,
                                       hop_length=512,
                                       fft_length=1024,
                                       n_mels=128)

def transform(mel_spectrogram,
              n_mels=128,
              frames=2,
              downsample=True,
              input_dim=64):
    """
    transform mel spectrogram to a vector array.

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    
    # 01 calculate the number of dimensions
    
    dims = n_mels * frames

    
    # 02 calculate total vector size
    
    vector_array_size = len(mel_spectrogram[0, :]) - frames + 1
    
    
    # 03 skip too short clips (fail safe)
    
    if vector_array_size < 1:
        print('recording too short!')
        return np.zeros(1, input_dims)
    
    # 04 generate feature vectors by concatenating multiframes (downsample mel spectrogram)
    
    if downsample:
        new_mels = 32
        new_frames = int(input_dim / new_mels)
        increment = int(n_mels / new_mels)  # value by which to sample the full 128 mels for each discrete time instant in each frame.
        offset = n_mels - new_mels * increment  # ensures all vector arrays have equal size
        assert (input_dim % new_mels == 0)

        vector_array = np.zeros((vector_array_size, new_mels * new_frames))

        for t in range(new_frames):
            new_vec = mel_spectrogram[:, t: t + vector_array_size].T
            vector_array[:, new_mels * t: new_mels * (t + 1)] = new_vec[:, offset::increment]

        return vector_array
    else:
        vector_array = np.zeros((vector_array_size, dims))
        for t in range(frames):
            vector_array[:, n_mels * t: n_mels * (t + 1)] = mel_spectrogram[:, t: t + vector_array_size].T
        return vector_array


def pred(data):
    
    predictions = np.zeros(np.shape(data))
    predictions[0, :] = nn.predict(data[0, :])
    error = np.mean(np.square(data - predictions), axis=1)
    return error


def score(score_buffer):

    y_pred = np.mean(score_buffer)
    # just print score for now
    print(y_pred, end='\r')
    
'''  
    if y_pred < 446.041404:
        print("normal,    anomaly score: %f" %y_pred, end="\r")
    else:
        print("anomalous, anomaly score: %f" %y_pred, end="\r")
'''


def capture(buffer, i):
    '''
    capture function 
    
        captures 1 frame and store in rolling buffer every 0.0000625s (16000 Hz)
    
    args:
    buffer = rolling buffer containing last 1024 frames
    '''
    overlay.audio.record(i/48000)
    buffer[:-i] = buffer[i:]
    for j in range(i):
        if overlay.audio.buffer[-j] > 2**23:
            buffer[-j] = (float) (overlay.audio.buffer[-j] - 2**24) / 2**24
        else:
            buffer[-j] = (float) overlay.audio.buffer[-j] / 2**24

    
def update(buffer, spectrogram_buffer, score_buffer, init_spectrogram, init_score):
    '''
    continuous processing function
    
        processes data in buffer and updates spectrogram every 0.032s (31.25 Hz)
        if spectrogram is filled, pass spectrogram through nn and print anomaly score

    args:
    buffer = rolling buffer containing last 1024 frames
    spectrogram = rolling mel spectrogram with last 2 time bins
    score_buffer = rolling buffer containing last 196 time slice mse
    init_spectrogram = number of time slices filled in mel spectrogram
    init_score = number of time slices filled in score_buffer
    
    '''
    # processes data in buffer
    temp_buffer = buffer
    temp = prep(temp_buffer)
    
    # updates spectrogram
    spectrogram_buffer[:, :-1] = spectrogram_buffer[:, 1:]
    spectrogram_buffer[:, -1:] = temp
    
    # only update score buffer if spectrogram is full
    if init_spectrogram < 2:
        init_spectrogram[0] = init_spectrogram[0] + 1
        return None
    
    temp_spectrogram = spectrogram_buffer
    data = transform(temp_spectrogram)
    score_buffer[:-1] = score_buffer[1:]
    score_buffer[-1:] = pred(data)
    
    # only calculate anomaly score if score_buffer is full
    if init_score < 196:
        init_score[0] = init_score[0] + 1
        return None
    
    temp_score = score_buffer
    score(temp_score)

In [ ]:
# testing live inference with threading

# this implementation will first fill up buffer. 
# once buffer is filled up, then audio data in buffer will be processed to filled up spectrogram_buffer
# once spectrogram_buffer is filled up, the spectrogram will be pass to the NN and the result is stored in score_buffer
# then score_buffer is used to calculate the anomaly score

def live_inference(seconds):
    # start initializing sequence
    print('initializing...')
    init_spectrogram = np.zeros(1)
    init_score = np.zeros(1)
    buffer = np.zeros(1024)
    spectrogram_buffer = np.zeros([128, 2])
    score_buffer = np.zeros(196)

    #filling buffer
    capture(buffer, 1024)

    # create timer object that calls on capture every 0.0000625s (16000 Hz)
    rt0 = RepeatedTimer(0.0000625, capture, buffer, 1)
    # create timer object that calls on update every 0.032s (31.25 Hz)
    rt1 = RepeatedTimer(0.032, update, buffer, spectrogram_buffer, score_buffer, init_spectrogram, init_score)

    try:
        sleep(seconds)
    finally:
        rt0.stop() 
        rt1.stop()

live_inference(50)

#### Awesome! now we can do live anomaly detection!